# Table 3.

Global top-of-atmosphere shortwave ($CRE_{SW}$), longwave ($CRE_{LW}$), and total ($CRE$) cloud radiative effect ($W~m^{-2}$) in the control and sensitivity simulations of Trappist-1e and Proxima b.

[Skip code and jump to the table](#Show-the-table)

----------------------------------

Import the necessary libraries.

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
from aeolus.calc import last_year_mean, toa_cloud_radiative_effect
from aeolus.core import Run

In [4]:
from commons import (
    GLM_MODEL_TIMESTEP,
    PLANET_ALIASES,
    RUN_ALIASES,
    output_name_prefix,
)
import mypaths

## Load data

Create a dictionary of `Run` objects with preprocessed data.

In [5]:
runs = {}
for planet in PLANET_ALIASES.keys():
    for run_key in RUN_ALIASES.keys():
        label = f"{planet}_{run_key}"
        
        fname = mypaths.sadir / label / "_processed" / f"{label}.nc"
        
        runs[label] = Run(
            files=fname, name=label, planet=planet, timestep=GLM_MODEL_TIMESTEP
        )

## Calculate global CRE

In [6]:
diags = {
    "$CRE_{SW}$": lambda cl: toa_cloud_radiative_effect(cl, kind="sw"),
    "$CRE_{LW}$": lambda cl: toa_cloud_radiative_effect(cl, kind="lw"),
    "$CRE$": lambda cl: toa_cloud_radiative_effect(cl, kind="total"),
}

Loop over planets and runs and store results in a `DataFrame`.

In [7]:
results = {}
for planet in PLANET_ALIASES.keys():
    dfs = {}
    for run_key in RUN_ALIASES.keys():
        label = f"{planet}_{run_key}"

        _data = {}
        for vrbl, _calc in diags.items():
            _data[vrbl] = float(last_year_mean(_calc(runs[label].proc)).data)
        dfs[run_key] = pd.DataFrame(
            _data, index=pd.Index(name="Simulation", data=[run_key])
        )
    results[planet] = pd.concat(dfs, axis="index").droplevel(0).T
df = pd.concat(results).T
df = df.rename(index=RUN_ALIASES).rename(columns=PLANET_ALIASES)

Print the `DataFrame` to LaTeX.

In [8]:
print(
    df.to_latex(
        formatters={k: lambda x: f"{x:.1f}" for k in df.columns},
        column_format="lcccccc",
        escape=False,
    )
)

\begin{tabular}{lcccccc}
\toprule
{} & \multicolumn{3}{l}{Trappist-1e} & \multicolumn{3}{l}{Proxima b} \\
{} &  $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ & $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ \\
Simulation &             &            &       &            &            &       \\
\midrule
MassFlux   &       -52.6 &       10.4 & -42.1 &      -58.5 &       15.7 & -42.8 \\
Adjust     &       -20.0 &        7.1 & -12.8 &      -20.1 &       10.5 &  -9.6 \\
NoCnvPm    &       -57.4 &       13.2 & -44.2 &      -63.9 &       13.5 & -50.4 \\
\bottomrule
\end{tabular}

